# User-Based Collaborative Filtering with Exploration

This notebook demonstrates how to evaluate a user-based collaborative filtering system that incorporates exploration of new movies. The system uses cosine similarity to find similar users and makes recommendations based on their ratings, while occasionally suggesting new movies to promote exploration.

### Import Required Libraries

In [1]:
from recsys.MovieLens import MovieLens
from surprise import KNNBasic
import heapq
import random
from collections import defaultdict
from operator import itemgetter
from recsys.RecommenderMetrics import RecommenderMetrics
from recsys.EvaluationDataset import EvaluationDataset

### Load Data

In [ ]:

lens, ratings_data, rankings = MovieLens.load()

### Prepare Evaluation Data

Set up the evaluation data using leave-one-out cross-validation (loo_iterator) to test our recommender system.

In [2]:
evaluation_dataset = EvaluationDataset(ratings_data, rankings)

# Train on leave-One-Out train set
loo_trainset = evaluation_dataset.loo_trainset
loo_testset = evaluation_dataset.loo_testset

### Train the Model

Train a KNN-based model using cosine similarity for user-based collaborative filtering.

In [3]:
sim_options = {
    'name': 'cosine',
    'user_based': True
}

model = KNNBasic(sim_options=sim_options)
model.fit(loo_trainset)
similarity_matrix = model.compute_similarities()

### Get New Movies for Exploration

Retrieve a list of new movies that we can use to promote exploration in our recommendations.

In [4]:
# Get new movies that need data
new_movies = lens.get_new_movies()
exploration_slot = 9  # Position in recommendations where we'll insert a new movie

### Generate Recommendations with Exploration

For each user, we'll:
1. Find their k most similar users
2. Get movies rated by similar users
3. Weight the ratings by user similarity
4. Insert a new movie at the exploration slot
5. Generate top-N recommendations

In [5]:
# Build up dict to lists of (int(movie_id), predictedrating) pairs
top_n = defaultdict(list)
k = 10  # Number of similar users to consider

for uiid in range(loo_trainset.n_users):
    # Get top N similar users to this one
    user_similarities = similarity_matrix[uiid]
    
    similar_users = []
    for inner_id, score in enumerate(user_similarities):
        if (inner_id != uiid):
            similar_users.append((inner_id, score))
    
    k_neighbours = heapq.nlargest(k, similar_users, key=lambda t: t[1])
    
    # Get the stuff they rated, and add up ratings for each item, weighted by user similarity
    candidates = defaultdict(float)
    for similar_user in k_neighbours:
        inner_id = similar_user[0]
        user_similarity_score = similar_user[1]
        users_own_ratings = loo_trainset.ur[inner_id]
        for rating in users_own_ratings:
            candidates[rating[0]] += (rating[1] / 5.0) * user_similarity_score
    
    # Build a dictionary of stuff the user has already seen
    watched = {}
    for item_id, rating in loo_trainset.ur[uiid]:
        watched[item_id] = 1
    
    # Get top-rated items from similar users:
    pos = 0
    for item_id, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
        if not item_id in watched:
            movie_id = 0
            if (pos == exploration_slot):
                movie_id = random.choice(new_movies)
            else:
                movie_id = loo_trainset.to_raw_iid(item_id)
            top_n[int(loo_trainset.to_raw_uid(uiid))].append((int(movie_id), 0.0))
            pos += 1
            if (pos > 40):
                break

### Evaluate Performance

Measure the hit rate of our recommendations to see how well the system performs with the exploration component.

In [6]:
hit_rate, cumulative_hit_rate, average_reciprocal_hit_rank, rating_hit_rate = RecommenderMetrics.hit_rate_metrics(top_n, loo_testset, 4.0)
 
(
    f"Hit Rate: {hit_rate}\n"
    f"Cumulative Hit Rate: {cumulative_hit_rate}\n"
    f"Average Reciprocal Hit Rank: {average_reciprocal_hit_rank}"
    f"Rating Hit Rate: {rating_hit_rate}"
)
